## Building a mobile spam filter with the multinomial Naive Bayes algorithm

To classify SMS messages as `spam` or `non-spam`, a computer:

- Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values: if the probability for `spam` is greater, then it classifies the message as `spam`. Otherwise, it classifies it as `non-spam` (if the two probability values are equal, then we may need a human to classify the message).

With this into account, our task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where it can also be found some of the authors' papers.

### Exploring the dataset

In [1]:
import pandas as pd

sms_df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
rows, columns = sms_df.shape
print('Nr rows: {}.\tNr columns: {}'.format(rows, columns))
print('Nr of spam SMS: {}.\tNr of non-spam SMS: {}'.format((sms_df['Label'] == 'spam').sum(), (sms_df['Label'] == 'ham').sum()))
print('Percentage of spam SMS: {}.\tPercentage of non-spam SMS: {}'.format(round(((sms_df['Label'] == 'spam').sum() / len(sms_df)) * 100, 2), round(((sms_df['Label'] == 'ham').sum() / len(sms_df)) * 100, 2)))

Nr rows: 5572.	Nr columns: 2
Nr of spam SMS: 747.	Nr of non-spam SMS: 4825
Percentage of spam SMS: 13.41.	Percentage of non-spam SMS: 86.59


### Building the spam filter: creating train and test sets

When creating software, a good rule of thumb is that designing the test comes before creating the software. Once our spam filter is done, we'll need to test how good it is with classifying _new messages_. To test the spam filter, we're first going to split our dataset into two categories:

- A __training set__, which we'll use to "train" the computer how to classify messages.
- A __test set__, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing.

In [2]:
# Training datatest
train_sms = sms_df.sample(frac=0.8, random_state=1)

# Testing datatest
test_sms = sms_df.sample(frac=0.2, random_state=1)

# Reseting index
train_sms.reset_index(drop=True, inplace=True)
test_sms.reset_index(drop=True, inplace=True)

# Checking outputs
print('train_sms shape: {}.\ntest_sms shape: {}'.format(train_sms.shape, test_sms.shape))

train_sms shape: (4458, 2).
test_sms shape: (1114, 2)


In [3]:
# Exploring train and test datasets
print('train_sms percentages:')
print((train_sms['Label'].value_counts(normalize=True)) * 100)
print('test_sms percentages:')
print((test_sms['Label'].value_counts(normalize=True)) * 100)

train_sms percentages:
ham     86.54105
spam    13.45895
Name: Label, dtype: float64
test_sms percentages:
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


The percentages obtained for both datasets are similar to the percentages obtained for the full dataset. Thus, we can assure the good aproximation to the data. We kept `random_state=1` for reproducibility reasons.

### Building the spam filter: cleaning the dataset

The next big step is to use the training set to teach the algorithm to classify new messages. When a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to the probability equations following the Bayes' Theorem.

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

The `SMS` column will be replaced by a series of new columns, where each column represents a unique word from the vocabulary. Each row describes a single message. All words in the vocabulary are in lower case and punctuation is not taken into account anymore.

In [4]:
# Before cleaning
print(train_sms['SMS'].head())

0                         Yep, by the pretty sculpture
1        Yes, princess. Are you going to make me moan?
2                           Welp apparently he retired
3                                              Havent.
4    I forgot 2 ask ü all smth.. There's a card on ...
Name: SMS, dtype: object


In [5]:
import re

def erase_punc(string_):
    return re.sub('\W', ' ', string_)

train_sms['SMS'] = train_sms['SMS'].apply(erase_punc).str.lower()

# After cleaning
print(train_sms['SMS'].head())

0                         yep  by the pretty sculpture
1        yes  princess  are you going to make me moan 
2                           welp apparently he retired
3                                              havent 
4    i forgot 2 ask ü all smth   there s a card on ...
Name: SMS, dtype: object


In [6]:
# Creating vocabulary
train_sms['SMS'] = train_sms['SMS'].str.split()
vocabulary = []
for row in train_sms['SMS']:
    for word in row:
        vocabulary.append(word)
vocabulary = set(vocabulary) # For removing duplicates
vocabulary = list(vocabulary)
print('Length of the train vocabulary: {}'.format(len(vocabulary)))

Length of the train vocabulary: 7783


In [7]:
# Creating dictionary with word frequencies
len_train = len(train_sms['SMS'])
word_counts_per_sms = {word: [0] * len_train for word in vocabulary}

for index, row in enumerate(train_sms['SMS']):
    for word in row:
        word_counts_per_sms[word][index] += 1

In [8]:
# Creating DataFrame from dictionary
word_counts = pd.DataFrame(word_counts_per_sms)
print(word_counts.head())

   0  00  000  000pes  008704050406  0089  01223585334  02  0207  02072069400  \
0  0   0    0       0             0     0            0   0     0            0   
1  0   0    0       0             0     0            0   0     0            0   
2  0   0    0       0             0     0            0   0     0            0   
3  0   0    0       0             0     0            0   0     0            0   
4  0   0    0       0             0     0            0   0     0            0   

  ...  zindgi  zoe  zogtorius  zouk  zyada  é  ú1  ü  〨ud  鈥  
0 ...       0    0          0     0      0  0   0  0    0  0  
1 ...       0    0          0     0      0  0   0  0    0  0  
2 ...       0    0          0     0      0  0   0  0    0  0  
3 ...       0    0          0     0      0  0   0  0    0  0  
4 ...       0    0          0     0      0  0   0  2    0  0  

[5 rows x 7783 columns]


In [9]:
# Resulting training dataset
train_sms_clean = pd.concat([train_sms, word_counts], axis=1)

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.

### Building the spam filter: calculating constants

As a start, let's first calculate:

- `P(Spam)`: probability a message is labelled as spam.
- `P(Ham)`: probability a message is labelled as non-spam.
- `NSpam`: the number of words in all the spam messages -not unique words-.
- `NHam`: the number of words in all the non-spam messages -not unique words-.
- `NVocabulary`: the number of unique words in all the messages.

We'll also use Laplace smoothing as additive smoothing and set α=1.

All of these values will be used to calculate the probabilities of new messages with same values, therefore they are called __constants__.

In [10]:
p_spam = (train_sms_clean['Label'] == 'spam').sum() / len(train_sms_clean)
p_ham = (train_sms_clean['Label'] == 'ham').sum() / len(train_sms_clean)
print('p_spam value: {}\np_ham value: {}'.format(p_spam, p_ham))

p_spam value: 0.13458950201884254
p_ham value: 0.8654104979811574


In [11]:
spam_sms = train_sms_clean[train_sms_clean['Label'] == 'spam'].copy()
ham_sms = train_sms_clean[train_sms_clean['Label'] == 'ham'].copy()
n_spam = spam_sms['SMS'].apply(len).sum() # SMS column is splitted
n_ham = ham_sms['SMS'].apply(len).sum() # SMS column is splitted
n_vocabulary = len(vocabulary)
print('n_spam: {}\nn_ham: {}\nn_vocabulary: {}'.format(n_spam, n_ham, n_vocabulary))

n_spam: 15190
n_ham: 57237
n_vocabulary: 7783


In [12]:
alpha = 1

### Building the spam filter: calculating parameters

We now can use our training set to calculate the probability for each word in our vocabulary. In more technical language, the probability values that `P(wi|Spam)` and `P(wi|Ham)` will take are called __parameters__.

In [13]:
# Initialization
parameters_spam = {word: 0 for word in vocabulary}
parameters_ham = {word: 0 for word in vocabulary}

# Function definition
def compute_parameters(dataset, vocabulary, alpha, n_dataset, n_vocabulary, params_dict):
    for word in vocabulary:
        nr_words = dataset[word].sum()
        parameter = (nr_words + alpha) / (n_dataset + (alpha * n_vocabulary))
        params_dict[word] = parameter

# Compute
compute_parameters(spam_sms, vocabulary, alpha, n_spam, n_vocabulary, parameters_spam)
compute_parameters(ham_sms, vocabulary, alpha, n_ham, n_vocabulary, parameters_ham)

### Creating the spam filter

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [14]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    print('P(Spam|message): {}\nP(Ham|message): {}'.format(p_spam_given_message, p_ham_given_message))

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
        

# Clasification examples
spam_message = "WINNER!! This is the secret code to unlock the money: C3421."
ham_message = "Sounds good, Tom, then see u there"
classify(spam_message)
classify(ham_message)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam
P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Testing the spam filter

We'll now try to determine how well the spam filter does on our test set of 1,114 messages. The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human).

We can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use __accuracy__ as a metric.

In [15]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

# Test algorithm
test_sms['predicted'] = test_sms['SMS'].apply(classify_test_set)
print(test_sms['predicted'].value_counts())

ham                           967
spam                          145
needs human classification      2
Name: predicted, dtype: int64


In [16]:
# Compute accuracy
correct = 0
total = len(test_sms)

for row in test_sms.itertuples(index=False):
    if row[0] == row[2]:
        correct += 1

accuracy = correct / total

# Outputs
print('Total test values: {}\nCorrect classified values: {}\nIncorrect classified values: {}'.format(total, correct, total - correct))
print('Test accuracy: {}'.format(accuracy))

Total test values: 1114
Correct classified values: 1106
Incorrect classified values: 8
Test accuracy: 0.992818671454219


The accuracy is close to 99.28%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,106 correctly and 8 incorrectly.